# Recommendation Engines: Content-based and Collaborative Filtering

## Content-Based

We will first build a content-based model using our movies from the classification problem, to find which movies are similiar to other movies based solely on the plot summary of the movie.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('movie_English_1960.csv', index_col=0)


In [3]:
df = df.drop_duplicates(subset='Title', keep="last")

In [4]:
df.reset_index(drop=True, inplace=True)

In [5]:
df.shape

(6706, 8)

In [6]:
df.tail(10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
6696,2012,The Most Fun You Can Have Dying,British,Kirstin Marcon,NaN,drama,https://en.wikipedia.org/wiki/The_Most_Fun_You...,"The story begins in Hamilton, New Zealands fou..."
6697,2012,Riot on Redchurch Street,British,Trevor Miller,NaN,crime,https://en.wikipedia.org/wiki/Riot_on_Redchurc...,"Set in the hipster-underworld of Shoreditch, E..."
6698,2013,Learning Hebrew,British,Louis Joon,NaN,drama,https://en.wikipedia.org/wiki/Learning_Hebrew,The plot notes on the UK version DVD sleeve read:
6699,2013,The Zombie King,British,Aidan Belizaire,NaN,comedy,https://en.wikipedia.org/wiki/The_Zombie_King,"Samuel Peters (Edward Furlong), once an ordina..."
6700,2014,Highway to Dhampus,British,Rick McFarland,NaN,drama,https://en.wikipedia.org/wiki/Highway_to_Dhampus,"Film Tagline:\r\nWe look at the sky, but we wa..."
6701,2014,Lost in Karastan,British,Ben Hopkins,NaN,comedy,https://en.wikipedia.org/wiki/Lost_in_Karastan,Lost in Karastan[2] is a gentle black comedy a...
6702,2014,One by One,British,Diane Jessie Miller,NaN,drama,https://en.wikipedia.org/wiki/One_by_One_(2013...,A cafe worker is violently jolted from her day...
6703,2014,The President,British,Mohsen Makhmalbaf,NaN,drama,https://en.wikipedia.org/wiki/The_President_(2...,A revolution is happening in a country with a ...
6704,2014,We Still Kill the Old Way (2014 film),British,Sacha Bennett,NaN,action,https://en.wikipedia.org/wiki/We_Still_Kill_th...,"A retired East End gangster, Ritchie Archer, r..."
6705,2014,What Goes Up,British,Matt Gambell,NaN,comedy,https://en.wikipedia.org/wiki/What_Goes_Up,"Upon arriving in Concord, New Hampshire in Jan..."


In [7]:
df.Genre.value_counts()

drama        2202
comedy       2144
horror        777
thriller      512
action        472
western       193
adventure     174
crime         125
romance       107
Name: Genre, dtype: int64

In [8]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(text):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(text)
    
#     mytokens = [word for word in mytokens if word.pos_ != "PROPN"]
    
    mytokens = [ word if word.pos_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [9]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(tokenizer=spacy_tokenizer, min_df=5, max_df=.7)


#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['Plot'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(6706, 15305)

With this matrix in hand, you can now compute a similarity score. There are several candidates for this; such as the euclidean, the Pearson and the cosine similarity scores. Again, there is no right answer to which score is the best. Different scores work well in different scenarios and it is often a good idea to experiment with different metrics.

You will be using the cosine similarity to calculate a numeric quantity that denotes the similarity between two movies. You use the cosine similarity score since it is independent of magnitude and is relatively easy and fast to calculate (especially when used in conjunction with TF-IDF scores, which will be explained later). Mathematically, it is defined as follows:


![alt text](https://miro.medium.com/max/2815/1*6HISTi8SjbD2VHicoZwKpA.png)

Since you have used the TF-IDF vectorizer, calculating the dot product will directly give you the cosine similarity score. Therefore, you will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.

In [10]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

You're going to define a function that takes in a movie title as an input and outputs a list of the 10 most similar movies. Firstly, for this, you need a reverse mapping of movie titles and DataFrame indices. 

In [11]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['Title'])

You are now in a good position to define your recommendation function. These are the following steps you'll follow:

- Get the index of the movie given its title.
- Get the list of cosine similarity scores for that particular movie with all movies. Convert it into a list of tuples where the first element is its position and the second is the similarity score.
- Sort the aforementioned list of tuples based on the similarity scores; that is, the second element.
- Get the top 10 elements of this list. Ignore the first element as it refers to self (the movie most similar to a particular movie is the movie itself).
- Return the titles corresponding to the indices of the top elements.

In [12]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['Title'].iloc[movie_indices]

In [13]:
get_recommendations('The Infidel')

5331           Grown Ups 2
3128      Mighty Aphrodite
835     The Heartbreak Kid
3645    Lulu on the Bridge
5065             Grown Ups
2069             The Boost
1187        Paradise Alley
6639            Incendiary
4067             Rock Star
3440           Eve's Bayou
Name: Title, dtype: object

In [14]:
get_recommendations('Learning Hebrew')

5950      Mister Ten Per Cent
2865        Totally F***ed Up
1564             Still Smokin
3687               Ringmaster
4847        Who's Your Caddy?
6053    Some Will, Some Won't
6660             London River
6665                  Telstar
4059    One Night at McCool's
2003    RuPaul Is: Starbooty!
Name: Title, dtype: object

In [15]:
get_recommendations('Grown Ups')

5331                          Grown Ups 2
3128                     Mighty Aphrodite
835                    The Heartbreak Kid
2069                            The Boost
6680                          The Infidel
1187                       Paradise Alley
6639                           Incendiary
4235    Dickie Roberts: Former Child Star
3624                            Happiness
3440                          Eve's Bayou
Name: Title, dtype: object

In [16]:
df.sample(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
2227,1989,Cutting Class,American,Rospo Pallenberg,"Donovan Leitch, Jill Schoelen, Brad Pitt, Mart...",horror,https://en.wikipedia.org/wiki/Cutting_Class,The plot revolves around the return of Brian W...
3369,1996,Twisted Desire,American,Craig R. Baxley,Melissa Joan Hart,drama,https://en.wikipedia.org/wiki/Twisted_Desire,Jennifer Stanton (Hart) is a rebellious teen w...
151,1962,The Three Stooges in Orbit,American,Edward Bernds,The Three Stooges,comedy,https://en.wikipedia.org/wiki/The_Three_Stooge...,The Stooges are TV actors who are trying to se...
4657,2006,Little Man,American,Keenen Ivory Wayans,"Marlon Wayans, Shawn Wayans, Kerry Washington",comedy,https://en.wikipedia.org/wiki/Little_Man_(2006...,"The film starts off with Calvin ""Babyface"" Sim..."
3188,1996,Alaska,American,Fraser Clarke Heston,"Thora Birch, Vincent Kartheiser, Dirk Benedict...",adventure,https://en.wikipedia.org/wiki/Alaska_(1996_film),Jake Barnes (Dirk Benedict) is flying a plane ...
1392,1981,Ragtime,American,Miloš Forman,"Elizabeth McGovern, Howard Rollins, James Cagn...",drama,https://en.wikipedia.org/wiki/Ragtime_(film),"The film begins with a newsreel montage, depic..."
5821,1963,Incident at Midnight,British,Norman Harrison,"Anton Diffring, William Sylvester",crime,https://en.wikipedia.org/wiki/Incident_at_Midn...,"Old Dr. Schroeder (Martin Miller), who has bee..."
4855,2008,Appaloosa,American,Ed Harris,"Ed Harris, Viggo Mortensen, Renée Zellweger, J...",western,https://en.wikipedia.org/wiki/Appaloosa_(film),"In 1882, the small town of Appaloosa, New Mexi..."
1889,1987,84 Charing Cross Road,American,David Jones,"Anne Bancroft, Anthony Hopkins, Judi Dench",drama,https://en.wikipedia.org/wiki/84_Charing_Cross...,"In 1949, Helene Hanff is in search of obscure ..."
750,1971,Duel,American,Steven Spielberg,"Dennis Weaver, Jacqueline Scott",thriller,https://en.wikipedia.org/wiki/Duel_(1971_film),David Mann is a middle-aged salesman driving o...


# Collaborative Filtering

Above we were using a content-based approach, which requires a good amount of information of items’ own features. Collaborative Filtering, on the other hand, doesn’t need anything else except users’ historical preference on a set of items. Because it’s based on historical data, the core assumption here is that the users who have agreed in the past tend to also agree in the future. 

## Memory Based
The first category includes algorithms that are memory based, in which statistical techniques are applied to the entire dataset to calculate the predictions.

### Nearest Neighborhood
The standard method of Collaborative Filtering is known as Nearest Neighborhood algorithm. There are user-based CF and item-based CF. 

#### User-based CF
![alt text](https://miro.medium.com/max/1813/1*mM089Lta5X6zkUkULcO9aA.png)

#### Item-based CF
![alt text](https://miro.medium.com/max/2048/1*dPzd5-dScFplypBGeSwgUw.png)

We are going to build an item-based collaborative filtering model to predict movie ratings for different users.  We have an n × m matrix of ratings, with user uᵢ, i = 1, ...n and item pⱼ, j=1, …m. Now we want to predict the rating rᵢⱼ if target user i did not watch/rate an item j. 
![alt text](https://files.realpython.com/media/rating-matrix.04153775e4c1.jpg)

## Steps Involved in Collaborative Filtering:

- How do you determine which users or items are similar to one another?
- Given that you know which users are similar, how do you determine the rating that a user would give to an item based on the ratings of similar users?
- How do you measure the accuracy of the ratings you calculate?

![alt text](https://miro.medium.com/max/2395/1*mTRUakSIWmo9OX6D2HakWQ.png)

While different people may have different baselines when giving ratings, some people tend to give high scores generally, some are pretty strict even though they are satisfied with items. To avoid this bias, we can subtract each user’s average rating of all items when computing weighted average, and add it back for target user, shown as below.

![alt text](https://miro.medium.com/max/2505/1*gLbwJts3g_v2TbPRhFoNfA.png)

Two ways to calculate similarity are **Pearson Correlation** and **Cosine Similarity**.

![alt text](https://miro.medium.com/max/3140/1*Xvf2o6kE4VCuueMPikxZ_A.png)

![alt text](https://miro.medium.com/max/2815/1*6HISTi8SjbD2VHicoZwKpA.png)

There are quite a few limitations of this method. It doesn’t handle sparsity well when no one in the neighborhood rated an item that is what you are trying to predict for target user. Also, it’s not computational efficient as the growth of the number of users and products.

In [ ]:
! pip install scikit-surprise

In [17]:
from surprise import Dataset
from surprise import Reader

# This is the same data that was plotted for similarity earlier
# with one new user "E" who has rated only movie 1
ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))

# Loads Pandas dataframe
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)
# Loads the builtin Movielens-100k data
movielens = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/stephaniekendall/.surprise_data/ml-100k


In [18]:
from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items, user based when True
}
algo = KNNWithMeans(k= 20, min_k=5, sim_options=sim_options)
# k is the number of neighbors, 5 is the minimum number of users to look at

In [19]:
trainingSet = data.build_full_trainset()

algo.fit(trainingSet)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [20]:
prediction = algo.predict('E', 2)
prediction.est

3.75

### Tuning the Algorithm Parameters


In [21]:
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}


param_grid = {"k":[10,20,30,40],
            "sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd

## Model Based
The second category covers the Model based approaches, which involve a step to reduce or compress the large but sparse user-item matrix.

**Matrix factorization** can be seen as breaking down a large matrix into a product of smaller ones. This is similar to the factorization of integers, where 12 can be written as 6 x 2 or 4 x 3. In the case of matrices, a matrix A with dimensions m x n can be reduced to a product of two matrices X and Y with dimensions m x p and p x n respectively.

#### Algorithms for Matrix Factorization
One of the popular algorithms to factorize a matrix is the singular value decomposition (SVD) algorithm. SVD came into the limelight when matrix factorization was seen performing well in the Netflix prize competition. Other algorithms include PCA and its variations, NMF, and so on. Autoencoders can also be used for dimensionality reduction in case you want to use Neural Networks.